# Diabetic Retinopathy Detection - InceptionResNetV2 (Multistage Training)

This notebook reproduces the pipeline from the paper:
*Diabetic Retinopathy Detection Using Deep Learning Multistage Training Method* (2025).

Dataset: [Diabetic Retinopathy 224x224 (2019 Data)](https://www.kaggle.com/datasets/sovitrath/diabetic-retinopathy-224x224-2019-data)

Author: Generated via ChatGPT


In [25]:
# Install dependencies (Kaggle already has most)
!pip install -q tensorflow==2.10.0 scikit-image sklearn opencv-python


ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.10.0


In [26]:
import os, glob, shutil, hashlib
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from skimage.transform import warp_polar

TARGET_SIZE = (224,224)
NUM_CLASSES = 5
DATASET_ROOT = "/kaggle/input/diabetic-retinopathy-224x224-2019-data"
STANDARD_ROOT = os.path.join(DATASET_ROOT, 'colored_images')


In [27]:
# Preprocessing helpers
def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def autocrop_black(img, tol=7):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.any():
        coords = np.argwhere(mask)
        y0,x0 = coords.min(axis=0)
        y1,x1 = coords.max(axis=0)
        return img[y0:y1+1, x0:x1+1]
    return img

def circular_crop(img):
    h,w = img.shape[:2]
    center = (w//2,h//2)
    radius = min(center[0],center[1],w-center[0],h-center[1])
    Y,X = np.ogrid[:h,:w]
    mask = (X-center[0])**2+(Y-center[1])**2 <= radius**2
    out = img.copy()
    out[~mask] = 0
    return out

def preprocess_image_file(path):
    img = load_image(path)
    img = autocrop_black(img)
    img = circular_crop(img)
    img = cv2.resize(img, TARGET_SIZE)
    return img


In [32]:
class_map = {
    "No DR": 0,
    "Mild": 1,
    "Moderate": 2,
    "Severe": 3,
    "Proliferative DR": 4
}

def make_file_list_and_labels(root_dir):
    classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir,d))]
    files, labels = [], []
    for c in classes:
        c_clean = c.strip()  # remove leading/trailing spaces
        if c_clean not in class_map:
            print(f"Skipping unknown folder: '{c}'")  # debug
            continue
        class_label = class_map[c_clean]
        for p in glob.glob(os.path.join(root_dir, c, '*')):
            files.append(p)
            labels.append(class_label)
    return files, labels, classes

files, labels, classes = make_file_list_and_labels(STANDARD_ROOT)


Skipping unknown folder: 'Proliferate_DR'
Skipping unknown folder: 'No_DR'


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(files, labels, test_size=0.2, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [35]:
AUTOTUNE = tf.data.AUTOTUNE
def path_label_to_dataset(paths, labels, batch_size=32, shuffle=True):
    paths = tf.constant(paths)
    labels = tf.constant(labels)
    ds = tf.data.Dataset.from_tensor_slices((paths,labels))
    if shuffle:
        ds = ds.shuffle(len(paths))
    def _load(path,label):
        img = tf.numpy_function(lambda p: preprocess_image_file(p.decode()), [path], tf.uint8)
        img = tf.cast(img, tf.float32)/255.0
        return img,label
    ds = ds.map(_load,num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = path_label_to_dataset(X_train,y_train)
val_ds = path_label_to_dataset(X_val,y_val)
test_ds = path_label_to_dataset(X_test,y_test)


In [36]:
# Build InceptionResNetV2 with custom head
base_model = tf.keras.applications.InceptionResNetV2(include_top=False,weights='imagenet',input_shape=(224,224,3))
x = base_model.output
x = layers.AveragePooling2D(pool_size=(5,5))(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.BatchNormalization()(x)
out = layers.Dense(NUM_CLASSES,activation='softmax')(x)
model = models.Model(inputs=base_model.input,outputs=out)


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [38]:
import tensorflow as tf

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    STANDARD_ROOT,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    STANDARD_ROOT,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 3662 files belonging to 5 classes.
Using 2930 files for training.
Found 3662 files belonging to 5 classes.
Using 732 files for validation.


In [39]:
# Stage 1: freeze base
for l in base_model.layers:
    l.trainable = False
model.compile(optimizer=optimizers.Adam(1e-3),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history1 = model.fit(train_ds,validation_data=val_ds,epochs=25)

# Stage 2: fine-tune from layer 100
for i,l in enumerate(base_model.layers):
    if i>=100: l.trainable=True
model.compile(optimizer=optimizers.Adam(1e-5),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history2 = model.fit(train_ds,validation_data=val_ds,epochs=25)


Epoch 1/25


I0000 00:00:1759455107.616961     109 service.cc:148] XLA service 0x7f0aa0004540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759455107.617766     109 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1759455112.251310     109 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/92 ━━━━━━━━━━━━━━━━━━━━ 58:25 39s/step - accuracy: 0.1875 - loss: 2.0593

I0000 00:00:1759455119.886309     109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


92/92 ━━━━━━━━━━━━━━━━━━━━ 79s 448ms/step - accuracy: 0.3813 - loss: 1.7068 - val_accuracy: 0.6066 - val_loss: 1.1861
Epoch 2/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.5862 - loss: 1.2146 - val_accuracy: 0.6612 - val_loss: 0.9629
Epoch 3/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6150 - loss: 1.1383 - val_accuracy: 0.6626 - val_loss: 0.9540
Epoch 4/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6045 - loss: 1.1409 - val_accuracy: 0.6694 - val_loss: 0.9215
Epoch 5/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.6184 - loss: 1.0824 - val_accuracy: 0.6639 - val_loss: 0.9234
Epoch 6/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6273 - loss: 1.0749 - val_accuracy: 0.6803 - val_loss: 0.9077
Epoch 7/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6225 - loss: 1.0493 - val_accuracy: 0.6776 - val_loss: 0.8760
Epoch 8/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.6234 - loss: 1.0587 - val_accuracy: 0.679

In [43]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    STANDARD_ROOT,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224,224),
    batch_size=32
)


Found 3662 files belonging to 5 classes.
Using 732 files for validation.


In [44]:
# Evaluate
print(model.evaluate(test_ds))
y_true,y_pred=[],[]
for x,y in test_ds:
    p = np.argmax(model.predict(x),axis=1)
    y_true.extend(y.numpy().tolist())
    y_pred.extend(p.tolist())
print(classification_report(y_true,y_pred))
#model.save("inceptionresnetv2_dr_multistage.h5")


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.7412 - loss: 0.8407
[0.8426254987716675, 0.744535505771637]
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 